In [1]:
### Notebook for plotting graphics from polygons

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
from shapely.geometry import MultiPolygon
from bs4 import BeautifulSoup

In [17]:
# ## New try # 1000 using chat gpt
# import pandas as pd
# from geopy import distance
# from math import radians

# # Load the data
# df = pd.read_csv("https://raw.githubusercontent.com/JSmith1826/BB_parks/main/data/homeplate_outbounds.csv")

# # Define a function to calculate the distance from home plate to a point given a degree
# def calc_distance(degree):
#     # Get the latitude and longitude of home plate
#     lat = df["homeplate"].values[0]
#     lon = df["homeplate"].values[1]
#     # Calculate the latitude and longitude of the point given a degree
#     point_lat = lat + (0.00000555556 * distance.feet(latitude=lat, longitude=lon).miles * radians(degree))
#     point_lon = lon + (0.00000555556 * distance.feet(latitude=lat, longitude=lon).miles * radians(degree) / abs(distance.lon_diff(latitude=lat, longitude=lon, miles=True)))
#     # Calculate the distance between home plate and the point
#     point = (point_lat, point_lon)
#     distance_feet = distance.distance(point, (lat, lon)).feet
#     return distance_feet

# # Calculate the distance for each degree in a 360 degree arc and store the results in a new column
# df["distance_feet"] = [calc_distance(degree) for degree in range(360)]

# # Display the dataframe
# print(df.head())


AttributeError: module 'geopy.distance' has no attribute 'feet'

In [24]:
import pandas as pd
from geopy.distance import distance
from math import radians

df = pd.read_csv("data\homeplate_outbound.csv")

# Function to calculate distance in feet
def calc_distance(lat, lon, lat2, lon2):
    dist_miles = distance((lat, lon), (lat2, lon2)).miles
    dist_feet = dist_miles * 5280
    return dist_feet

# Calculate distance for each degree in a 360 degree arc
for degree in range(0, 360):
    distances = []
    for index, row in df.iterrows():
        lat = row["homeplate"].split(", ")[0]
        lon = row["homeplate"].split(", ")[1]
        poly = row["poly"][1:-1] # Remove parentheses from string
        lat2 = poly.split(", ")[0]
        lon2 = poly.split(", ")[1]
        distance_feet = calc_distance(float(lat), float(lon), float(lat2), float(lon2))
        distances.append(distance_feet)
    df[f"distance_{degree}_feet"] = distances


ValueError: could not convert string to float: '(-86.2258505'

In [11]:
#### Load data from kml file exported by Google Earth

file_path = ('data/kml/ballparks.kml')

with open(file_path) as file:

    xml_data = file.read()



# Initialize soup variables for parsing file
soup = BeautifulSoup(xml_data, 'xml')

folders = soup.Document.Folder
list = soup.Document.Folder.find_all('Folder')
# layers = soup.Document.Folder.Folder
# polygons = soup.Document.Folder.Placemark.Polygon

## Create a dataframe to hold the data parsed from xml
df = pd.DataFrame(columns=['field', 'foul', 'fop'])


## Loop through the folders and extract the data
i = 0   

for i in range(len(list)):

    folders = list[i]
    field_name = folders.find('name').text
    foul = folders.find_all('coordinates')[0].text
    fop = folders.find_all('coordinates')[1].text

    row = {
        'field': field_name,
        'foul': foul,
        'fop': fop
    }

    i+=1

    df = df.append(row, ignore_index=True)

## Cleaning
# remove new line and and space characters from coordinates
df = df.replace(r'\n','', regex=True) 
df = df.replace(r'\t','', regex=True) 

## Drop any duplicate rows
df = df.drop_duplicates(subset=['field'], keep='first')

## Drop any rows with empty fields
df = df[(df != 0).all(1)]

# ## remove any numberic characters and . from field names
# # There shouldn't be any anymore because of the validation I did before exporting kml
# df['field'] = df['field'].str.replace(r'\d+', '')

## Parse field names to get level column using regex
import re
re_mlb = re.compile(r'mlb', re.IGNORECASE)
re_pro = re.compile(r'pro', re.IGNORECASE)
re_college = re.compile(r'college', re.IGNORECASE)
re_youth = re.compile(r'youth', re.IGNORECASE)
re_muni = re.compile(r'muni', re.IGNORECASE)

df['level'] = df['field'].apply(lambda x: 'mlb' 
        if re_mlb.search(x) else 'pro' 
        if re_pro.search(x) else 'college' 
        if re_college.search(x) else 'youth' 
        if re_youth.search(x) else 'muni' 
        if re_muni.search(x) else 'high_school')

# clean up the field names
# remove the level from the field name
df['field'] = df['field'].str.replace(r'MLB', '')
df['field'] = df['field'].str.replace(r'pro', '')
df['field'] = df['field'].str.replace(r'college', '')
df['field'] = df['field'].str.replace(r'High School', 'HS')
# remove - from end of field name
df['field'] = df['field'].str.replace(r'- $', '')

## Output test csv
# df.to_csv('TEMP/level2_tost.csv', index=False)

df['foul_poly'] = df['foul'].apply(lambda x: {'type': 'Polygon', 'coordinates': [[tuple(map(float, coord.split(','))) for coord in x.split()]]})
df['fop_poly'] = df['fop'].apply(lambda x: {'type': 'Polygon', 'coordinates': [[tuple(map(float, coord.split(','))) for coord in x.split()]]})

C:\Users\Justin\AppData\Local\Temp\ipykernel_21312\3853256035.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\Justin\AppData\Local\Temp\ipykernel_21312\3853256035.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\Justin\AppData\Local\Temp\ipykernel_21312\3853256035.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\Justin\AppData\Local\Temp\ipykernel_21312\3853256035.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\Justin\AppData\Loca

In [3]:
## get homeplate coordinates

df['fop_hp'] = df['fop'].apply(lambda x: tuple(map(float, x.split(',')[0:2])))

In [4]:
#### Merge the foul_poly and fop_poly columns into a single polygon column

import shapely.geometry as geom
import shapely.ops as ops
from shapely import affinity

df['poly'] = df.apply(lambda x: ops.unary_union([geom.shape(x['foul_poly']), geom.shape(x['fop_poly'])]), axis=1)

### Drop unnecessary columns

df = df.drop(columns=['foul', 'fop', 'foul_poly', 'fop_poly'])

## Rename fop_hp column to homeplate
df = df.rename(columns={'fop_hp': 'homeplate'})

c:\Users\Justin\anaconda3\lib\site-packages\pandas\core\dtypes\cast.py:1981: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  result[:] = values


In [7]:
df.head()

## write a csv
df.to_csv('data/homeplate_outbound.csv', index=False)



In [6]:
# ##This script first defines a haversine function to calculate the distance between two points on a globe. It then defines a calculate_distances function that takes the latitude and longitude of home plate, and a polygon that represents the outer boundaries of the park. It calculates the distance from home plate to the outer boundary for each degree in a 360 degree arc, and returns the distances as a dictionary.

# ## The script then defines an example DataFrame with the latitude and longitude of home plate, and an example polygon that represents the outer boundaries of the park. It calls the calculate_distances function to calculate the distances, and prints them out for each degree in a 360 degree arc.

# import pandas as pd
# import numpy as np
# from shapely.geometry import Point, Polygon

# # define function to calculate distance between two points
# def haversine(lat1, lon1, lat2, lon2):
#     # convert decimal degrees to radians
#     lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])

#     # haversine formula
#     dlat = lat2 - lat1
#     dlon = lon2 - lon1
#     a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
#     c = 2 * np.arcsin(np.sqrt(a))
#     km = 6371 * c
#     return km

# # define function to calculate distance from home plate to outer boundary for each degree in a 360 degree arc
# def calculate_distances(home_plate_lat, home_plate_lon, polygon):
#     distances = {}
#     for degree in range(0, 360):
#         # calculate endpoint coordinates for given degree
#         endpoint_lat = home_plate_lat + np.sin(np.radians(degree))
#         endpoint_lon = home_plate_lon + np.cos(np.radians(degree))

#         # calculate distance from home plate to endpoint
#         endpoint = Point(endpoint_lat, endpoint_lon)
#         distance = polygon.boundary.distance(endpoint)

#         # add distance to dictionary
#         distances[degree] = distance

#     return distances

# # define example DataFrame with latitude and longitude of home plate
# df = pd.DataFrame({'latitude': [37.7786], 'longitude': [-122.3892]})

# # define example polygon that represents outer boundaries of park
# polygon_coords = [(37.7781, -122.3893), (37.7781, -122.3888), (37.7789, -122.3888), (37.7789, -122.3893)]
# polygon = Polygon(polygon_coords)

# # calculate distance from home plate to outer boundary for each degree in a 360 degree arc
# distances = calculate_distances(df['latitude'][0], df['longitude'][0], polygon)

# # print distances
# for degree, distance in distances.items():
#     print(f'{degree} degrees: {distance} meters')


In [7]:
# import pandas as pd
# from shapely.geometry import Point, Polygon

# # Read in the data from a CSV file
# df = pd.read_csv('data/homeplate_outbounds.csv')

# # Convert homeplate column to Point objects
# df['homeplate'] = df['homeplate'].apply(lambda x: Point(eval(x)))

# # Convert poly column to Polygon objects
# df['poly'] = df['poly'].apply(lambda x: Polygon([tuple(map(float, t.split()))[:2] for t in x.split('((')[1].split('))')[0].split(',')]))

# # Rename columns to match the variable names in the code block above
# df = df.rename(columns={'homeplate': 'center', 'poly': 'polygon'})

# # Print the processed DataFrame
# print(df)


In [8]:
## Rename poly to polygon
df = df.rename(columns={'poly': 'polygon'})

In [9]:
from math import radians, cos, sin, sqrt

def calc_distance(lat1, lon1, lat2, lon2):
    R = 20925604.7 # radius of the earth in feet
    phi1 = radians(lat1)
    phi2 = radians(lat2)
    d_phi = radians(lat2-lat1)
    d_lambda = radians(lon2-lon1)
    a = sin(d_phi/2)**2 + cos(phi1)*cos(phi2)*sin(d_lambda/2)**2
    d = 2*R*sqrt(a)
    return d

from shapely.geometry import Polygon, MultiPolygon

def get_distance_to_polygon(homeplate, polygon):
    homeplate_lat, homeplate_lon = homeplate.strip('()').split(', ')
    homeplate_lat = float(homeplate_lat)
    homeplate_lon = float(homeplate_lon)
    if isinstance(polygon, Polygon):
        polygon = polygon.exterior
    elif isinstance(polygon, MultiPolygon):
        polygon = polygon[0].exterior
    polygon = str(polygon)
    polygon = polygon.strip('POLYGON Z ()')
    polygon = polygon.replace(' ',',')
    polygon_coords = polygon.split(',')
    polygon_coords = [float(coord) for coord in polygon_coords if coord != '']
    n = len(polygon_coords)
    distances = []
    for i in range(0, 360):
        angle = radians(i)
        min_distance = float('inf')
        for j in range(0, n-3, 2):
            lat1, lon1, lat2, lon2 = polygon_coords[j:j+4]
            distance = calc_distance(homeplate_lat, homeplate_lon, lat1, lon1)
            distance += calc_distance(lat1, lon1, lat2, lon2)*angle
            distance += calc_distance(lat2, lon2, homeplate_lat, homeplate_lon)
            min_distance = min(min_distance, distance)
        distances.append(min_distance)
    return distances



In [10]:
from shapely.geometry import Polygon, Point

def get_distance_to_polygon(row):
    polygon = row['polygon'].replace('POLYGON Z ((', '').replace('))', '')
    polygon_coords = [tuple(map(float, x.split()))[:2] for x in polygon.split(',')]
    point = Point(row['longitude'], row['latitude'])
    polygon = Polygon(polygon_coords)
    return point.distance(polygon)


In [11]:
df_distances = df.apply(get_distance_to_polygon, axis=1)
df_distances.columns = [f'Distance_{i}' for i in range(360)]
df = pd.concat([df, df_distances], axis=1)


AttributeError: 'Polygon' object has no attribute 'replace'

In [10]:
df.head()

,field,level,homeplate,polygon
0,Frankfort-Elberta Area HS,high_school,"(-86.2258505, 44.6344082)","POLYGON Z ((-86.226006 44.6336304 0, -86.22610..."
1,Ironwood HS,high_school,"(-90.1514578, 46.457154)","POLYGON Z ((-90.1506035 46.4565313 0, -90.1506..."
2,Oakland A's - mlb,mlb,"(-122.2010782, 37.7513245)","POLYGON Z ((-122.1999704 37.7511479 0, -122.19..."
3,Citi Field - NY Mets - mlb,mlb,"(-73.8460277, 40.7565867)","POLYGON Z ((-73.8450072 40.7570454 0, -73.8453..."
4,Citizens Bank Park - Philadelphia Phillies - mlb,mlb,"(-75.1665943, 39.9055805)","POLYGON Z ((-75.1656247 39.9060907 0, -75.1656..."
